#🔬 **Bioactivity Classification of EGFR Inhibitors Using Machine Learning**

#Introduction

In the evolving landscape of drug discovery, one of the most pressing challenges is the time and cost associated with identifying potential drug candidates. Traditional methods often involve years of laboratory work, extensive biological screening, and a high attrition rate. To accelerate this pipeline, computer-aided drug discovery (CADD) has emerged as a vital strategy, enabling researchers to use data-driven methods to prioritize and evaluate chemical compounds before experimental validation.

Within CADD, machine learning (ML) plays a central role, learning patterns from historical bioactivity data to predict the potential of unknown molecules. This project applies ML to tackle a fundamental problem in cheminformatics: predicting whether a compound is likely to be biologically active against a specific therapeutic target.

My focus is the Epidermal Growth Factor Receptor (EGFR), a protein frequently overexpressed or mutated in cancers. I aim to classify small molecules as either active or inactive EGFR inhibitors, using curated chemical and bioactivity data from the ChEMBL database.

#What is EGFR?

EGFR (Epidermal Growth Factor Receptor) is a receptor tyrosine kinase involved in the regulation of cell division, survival, and proliferation. Genetic mutations or dysregulation of EGFR signaling are implicated in various cancers, notably non-small cell lung cancer, glioblastoma, and colorectal cancer. EGFR is a validated drug target, and several small-molecule inhibitors (e.g., Gefitinib, Erlotinib) have been approved for cancer therapy.

Identifying new EGFR inhibitors can aid in the development of targeted cancer treatments and reduce off-target effects through rational compound design.

#What is ChEMBL?

The ChEMBL database is an open-access bioactivity resource curated by the European Bioinformatics Institute (EMBL-EBI). It provides detailed records of drug-like small molecules, their biological assay results, and associated molecular targets.

#As of ChEMBL version 33 (March 2025), the database contains:

1) 2.3+ million unique compounds

2) Bioactivity data from 82,000+ scientific publications

3) Results from 1.3 million bioassays

4) Information covering 14,000 molecular targets, 1,900 cell lines, and 34,500 disease indications

5) Its structured format makes it ideal for applying machine learning to real-world bioactivity prediction problems.

#Project Aim

This project aims to develop an ML-based classification pipeline for predicting the EGFR inhibitory activity of chemical compounds. The specific objectives include:

1) Preprocessing the ChEMBL-derived dataset by cleaning, deduplicating, and standardizing chemical entries.

2) Generating molecular fingerprints (Morgan/ECFP4) and physicochemical descriptors using RDKit.

3) Labeling compounds as active or inactive based on pIC50 thresholds.

4) Training and evaluating classification models to predict bioactivity.

5) Analyzing feature contributions to understand molecular substructures associated with inhibition.

By building a robust classification model, this project demonstrates how machine learning can assist in early-stage drug screening, enabling researchers to filter large libraries of compounds for further testing.

#**Installing Libraries**

Installing the ChEMBL web service package so that I can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.0 MB/s eta 0:00:00


#**Importing Libraries**

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

#**Searching for Target Protein (EFGR)**

In [ ]:
targets = new_client.target.filter(
    target_components__accession="P00533",
    target_type="SINGLE PROTEIN",
    organism="Homo sapiens"
)

df = pd.DataFrame.from_records(targets)
print(df[['target_chembl_id', 'pref_name', 'target_type', 'organism']])

  target_chembl_id                               pref_name     target_type  \
0        CHEMBL203  Epidermal growth factor receptor erbB1  SINGLE PROTEIN   
1        CHEMBL203  Epidermal growth factor receptor erbB1  SINGLE PROTEIN   

       organism  
0  Homo sapiens  
1  Homo sapiens  


This shows that the erbB1 protein is present in the database. Now I just have to extract the data.

#**Dataset Acquisition from ChEMBL**

To build a meaningful classification model, I needed a reliable and high-quality dataset of compounds with experimentally measured activity values against the target of interest — the Epidermal Growth Factor Receptor (EGFR).

The following code queries the ChEMBL database via its Python client to fetch compounds tested against EGFR using a well-defined and reproducible filtering strategy:

In [ ]:
chembl_id = "CHEMBL203"

activity = new_client.activity.filter(
    target_chembl_id=chembl_id,
    standard_type="IC50",
    relation="=",
    assay_type="B"
).only("molecule_chembl_id", "standard_value", "standard_units")

data = []
for i, rec in enumerate(activity):
    if i >= 2000:
        break
    data.append(rec)

df = pd.DataFrame(data)
print("Loaded rows:", len(df))
df.head()

Loaded rows: 2000


,molecule_chembl_id,standard_units,standard_value,units,value
0,CHEMBL68920,nM,41.0,uM,0.041
1,CHEMBL69960,nM,170.0,uM,0.17
2,CHEMBL137635,nM,9300.0,uM,9.3
3,CHEMBL306988,nM,500000.0,uM,500.0
4,CHEMBL66879,nM,3000000.0,uM,3000.0


#What is CHEMBL203?

This is the ChEMBL ID for the human EGFR protein the biological target for which I am retrieving compound activity data. I am focusing on a single, specific protein target to maintain the biological relevance and consistency of the dataset. Including multiple targets could introduce noise due to differing binding sites, isoforms, or mechanisms of action.

#Why IC50?

1) IC50 (half-maximal inhibitory concentration) is a widely used metric in drug discovery.

2) It represents the concentration of a compound required to inhibit 50% of its target’s biological activity.

3) IC50 is a continuous, quantitative measure of how potent a compound is.

4) A lower IC50 value indicates higher potency, i.e., the compound is effective at low concentrations.

I have filtered for standard_type = "IC50" because it's:

1) One of the most commonly reported and comparable bioactivity measures.

2) Ideal for classification (active vs. inactive) based on well-established thresholds.

#Why relation = "="?

The relation field in ChEMBL refers to how the IC50 value is reported (e.g., =, >, <, ~).
I am only using entries with relation = "=" to ensure that the reported IC50 values are exact and numerically comparable — essential for consistent labeling and thresholding in the classification task.

#Why assay_type = "B"?

ChEMBL classifies assays into several types.

1) "B" stands for binding assays, which measure the physical interaction between a compound and its target (e.g., EGFR).

2) Binding assays are generally more direct and reliable compared to functional assays ("F"), which measure downstream biological effects.

3) Using only binding assays helps ensure that the IC50 values reflect direct inhibition of EGFR, rather than effects mediated by other biological pathways.

#What is standard_value, and why is it important?

1) standard_value is a numerical representation of the IC50 (in standard units, typically nanomolar (nM)).

2) For example, if IC50 = 50 nM, then standard_value = 50 and standard_units = "nM".

I am using standard_value for classification because:

1) It provides a normalized and unified scale for comparing compound potencies.

2) It is the only numerical field I need to define compound activity classes (e.g., active if IC50 < 1000 nM).

#Why this filtering strategy?

By filtering on:

1) A single protein (CHEMBL203)

2) A standardized activity measure (IC50)

3) A precise relation (=)

4) A biologically direct assay type (B)

...I ensure that the dataset is:

1) Clean, with minimal ambiguity

2) Biologically consistent, targeting only EGFR

3) Suitable for supervised learning, especially binary classification (active/inactive)

Also, I have limited the dataset to the first 2000 entries for faster experimentation and prototyping, although this can be increased for production models.

Saving the dataset...

In [ ]:
df.to_csv("bioactivity_data.csv", index=False)

Checking for missing data

In [ ]:
df.isnull().sum()

,0
molecule_chembl_id,0
standard_units,0
standard_value,1
units,49
value,1


Now, the only column which will be useful for calculating SMILES later is the standard_value column. Since there is one missing value there, I need to remove that row. Null values in the units column are not important.

The below code is just for checking if standard_units have all the entries correctly.

In [ ]:
df.standard_units.value_counts()

,count
standard_units,
nM,2000


Everything's fine. Now removing the missing data row...

In [ ]:
df_clean = df.dropna(subset=["standard_value"])

In [ ]:
df_clean.isnull().sum()

,0
molecule_chembl_id,0
standard_units,0
standard_value,0
units,49
value,0


Data is cleaned.

#**Classification Criteria**:

- Active: IC50 ≤ 1000 nM

- Inactive: IC50 ≥ 10000 nM

- ntermediate: IC50 between 1000 and 10000 nM

These thresholds are commonly used in cheminformatics to separate clearly potent compounds (active) from weak or ineffective ones (inactive), while ignoring borderline compounds (intermediate) that may introduce noise in a classification task.

In later steps, I will discard the intermediate class and retain only active and inactive compounds to build a clean and unambiguous binary classification model.


In [ ]:
bioactivity_class = []
for i in df_clean.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

Final checking. Number of samples = 1999 (2000 - 1 missing standard value row)

In [ ]:
len(bioactivity_class)

1999

#Retrieving Canonical SMILES Representations

After filtering the dataset for bioactivity data, I needed to obtain the chemical structures of each compound in a machine-readable format. This is essential for computing molecular descriptors and fingerprints in the next steps.

The following code fetches the canonical SMILES for each molecule from ChEMBL:

In [ ]:
from tqdm import tqdm

molecule_client = new_client.molecule

smiles_list = []

for mol_id in tqdm(df_clean['molecule_chembl_id']):
    try:
        mol_data = molecule_client.get(mol_id)
        smiles = mol_data['molecule_structures']['canonical_smiles']
    except:
        smiles = None
    smiles_list.append(smiles)

df_clean['canonical_smiles'] = smiles_list

print(df_clean.head())

100%|██████████| 1999/1999 [09:07<00:00,  3.65it/s]

  molecule_chembl_id standard_units standard_value units   value  \
0        CHEMBL68920             nM           41.0    uM   0.041   
1        CHEMBL69960             nM          170.0    uM    0.17   
2       CHEMBL137635             nM         9300.0    uM     9.3   
3       CHEMBL306988             nM       500000.0    uM   500.0   
4        CHEMBL66879             nM      3000000.0    uM  3000.0   

                                    canonical_smiles  
0  Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...  
1  Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...  
2        CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12  
3             CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1  
4                             O=C(O)/C=C/c1ccc(O)cc1  



<ipython-input-39-2329285210>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['canonical_smiles'] = smiles_list


#What is SMILES?

SMILES (Simplified Molecular Input Line Entry System) is a line notation for representing molecular structures using short ASCII strings. It encodes the atoms and the connectivity (bonds) in a molecule.

For example:

- Ethanol → CCO

- Benzene → c1ccccc1

I specifically retrieved **canonical SMILES**, which are standardized representations generated deterministically. This ensures:

1) Consistency across all entries

2) Compatibility with RDKit for further descriptor and fingerprint generation

3) Avoidance of redundant molecular formats

SMILES are widely used in cheminformatics because they are:

- Compact and lightweight

- Easily parsed by programs

- Convertible to structural models (2D/3D) when needed

#Why did I use tqdm?

I used the tqdm library to display a progress bar while iterating through each compound’s ChEMBL ID. Since I was querying the ChEMBL API for hundreds of molecules, this helped me:

1) Track progress visually

2) Monitor how long the retrieval process was taking

3) Spot delays or any points where the process may have stalled

4) Without tqdm, the loop would run silently, offering no indication of how many entries had been processed or how much time remained.



This step added a new column, canonical_smiles to my dataset, which I will use next to generate structure-based molecular features for model training.

In [ ]:
df_clean.isnull().sum()

,0
molecule_chembl_id,0
standard_units,0
standard_value,0
units,49
value,0
canonical_smiles,0


In [ ]:
df_clean.shape

(1999, 6)

Dataset preparation with features and class-label column.

In [ ]:
data = pd.concat([df_clean.drop(['standard_units','units','value'], axis=1), pd.Series(bioactivity_class)], axis=1)

In [ ]:
data.head()

,molecule_chembl_id,standard_value,canonical_smiles,0
0,CHEMBL68920,41.0,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,active
1,CHEMBL69960,170.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,active
2,CHEMBL137635,9300.0,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,intermediate
3,CHEMBL306988,500000.0,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,inactive
4,CHEMBL66879,3000000.0,O=C(O)/C=C/c1ccc(O)cc1,inactive


In [ ]:
data.rename(columns={0: 'bioactivity_class'}, inplace=True)

In [ ]:
data.head()

,molecule_chembl_id,standard_value,canonical_smiles,bioactivity_class
0,CHEMBL68920,41.0,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,active
1,CHEMBL69960,170.0,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,active
2,CHEMBL137635,9300.0,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,intermediate
3,CHEMBL306988,500000.0,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,inactive
4,CHEMBL66879,3000000.0,O=C(O)/C=C/c1ccc(O)cc1,inactive


In [ ]:
data.isnull().sum()

,0
molecule_chembl_id,1
standard_value,1
canonical_smiles,1
bioactivity_class,1


One missing value is still there, and it needs to be removed.

In [ ]:
data = data.dropna()
data.isnull().sum()

,0
molecule_chembl_id,0
standard_value,0
canonical_smiles,0
bioactivity_class,0


In [ ]:
data.shape

(1998, 4)

To ensure that each compound in my dataset is unique, I removed duplicate entries based on their canonical SMILES:

In [ ]:
data = data.drop_duplicates(subset='canonical_smiles', keep='first').reset_index(drop=True)

This keeps only the first occurrence of each unique molecule and discards the rest, preventing biased model training due to repeated structures.

In [ ]:
data.shape

(1535, 4)

In [ ]:
data.duplicated(subset='canonical_smiles').sum()

np.int64(0)

No duplication now.

Finally, I saved the cleaned and annotated dataset to a CSV file for downstream analysis:

In [ ]:
data.to_csv("preprocessed_bioactivity_data.csv", index=False)

This file contains unique compounds with their ChEMBL IDs, IC50 values, bioactivity classes, and canonical SMILES, ready for feature extraction and model building in the next steps.